### Task 2.1

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader

import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 100
batch_size = 100
learning_rate = 0.0002

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

mnist = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True)

# Discriminator network
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
).to(device)

# Generator network
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(G.parameters(), lr=learning_rate)

# Function to reset gradients
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # Discriminator training
        # Real images
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Generator training
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Create directory if it doesn't exist
    os.makedirs('./gan_images', exist_ok=True)

    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        utils.save_image(images, './gan_images/real_images.png')
        utils.save_image(fake_images, './gan_images/fake_images-{}.png'.format(epoch+1))

# Save the model checkpoints
torch.save(G.state_dict(), './gan_generator.pth')
torch.save(D.state_dict(), './gan_discriminator.pth')


Epoch [0/100], Step [200/600], d_loss: 0.0536, g_loss: 3.9268, D(x): 0.99, D(G(z)): 0.05
Epoch [0/100], Step [400/600], d_loss: 0.1343, g_loss: 5.7755, D(x): 0.96, D(G(z)): 0.07
Epoch [0/100], Step [600/600], d_loss: 0.0405, g_loss: 6.2206, D(x): 0.98, D(G(z)): 0.02
Epoch [1/100], Step [200/600], d_loss: 0.0224, g_loss: 5.2519, D(x): 0.99, D(G(z)): 0.01
Epoch [1/100], Step [400/600], d_loss: 0.4978, g_loss: 3.1265, D(x): 0.86, D(G(z)): 0.22
Epoch [1/100], Step [600/600], d_loss: 0.0651, g_loss: 5.5196, D(x): 0.99, D(G(z)): 0.05
Epoch [2/100], Step [200/600], d_loss: 0.0629, g_loss: 4.2479, D(x): 0.97, D(G(z)): 0.02
Epoch [2/100], Step [400/600], d_loss: 0.3497, g_loss: 4.0526, D(x): 0.84, D(G(z)): 0.03
Epoch [2/100], Step [600/600], d_loss: 0.3176, g_loss: 4.4665, D(x): 0.95, D(G(z)): 0.18
Epoch [3/100], Step [200/600], d_loss: 0.6252, g_loss: 2.0339, D(x): 0.86, D(G(z)): 0.28
Epoch [3/100], Step [400/600], d_loss: 0.5804, g_loss: 2.5079, D(x): 0.92, D(G(z)): 0.27
Epoch [3/100], Step [

Epoch [30/100], Step [600/600], d_loss: 0.3952, g_loss: 3.1094, D(x): 0.94, D(G(z)): 0.19
Epoch [31/100], Step [200/600], d_loss: 0.2733, g_loss: 3.5169, D(x): 0.91, D(G(z)): 0.11
Epoch [31/100], Step [400/600], d_loss: 0.4956, g_loss: 3.1062, D(x): 0.86, D(G(z)): 0.17
Epoch [31/100], Step [600/600], d_loss: 0.5118, g_loss: 1.7379, D(x): 0.87, D(G(z)): 0.18
Epoch [32/100], Step [200/600], d_loss: 0.3279, g_loss: 3.2200, D(x): 0.95, D(G(z)): 0.19
Epoch [32/100], Step [400/600], d_loss: 0.4195, g_loss: 3.6069, D(x): 0.84, D(G(z)): 0.08
Epoch [32/100], Step [600/600], d_loss: 0.2367, g_loss: 4.5972, D(x): 0.90, D(G(z)): 0.05
Epoch [33/100], Step [200/600], d_loss: 0.3618, g_loss: 3.9266, D(x): 0.89, D(G(z)): 0.13
Epoch [33/100], Step [400/600], d_loss: 0.3472, g_loss: 3.8404, D(x): 0.89, D(G(z)): 0.13
Epoch [33/100], Step [600/600], d_loss: 0.2757, g_loss: 2.9219, D(x): 0.87, D(G(z)): 0.06
Epoch [34/100], Step [200/600], d_loss: 0.5219, g_loss: 3.7814, D(x): 0.83, D(G(z)): 0.12
Epoch [34/

Epoch [61/100], Step [400/600], d_loss: 0.6704, g_loss: 2.2064, D(x): 0.83, D(G(z)): 0.28
Epoch [61/100], Step [600/600], d_loss: 0.7455, g_loss: 3.0770, D(x): 0.70, D(G(z)): 0.13
Epoch [62/100], Step [200/600], d_loss: 0.6316, g_loss: 1.8840, D(x): 0.85, D(G(z)): 0.27
Epoch [62/100], Step [400/600], d_loss: 0.4358, g_loss: 2.0672, D(x): 0.87, D(G(z)): 0.19
Epoch [62/100], Step [600/600], d_loss: 0.5278, g_loss: 3.3879, D(x): 0.86, D(G(z)): 0.21
Epoch [63/100], Step [200/600], d_loss: 0.6840, g_loss: 3.0650, D(x): 0.75, D(G(z)): 0.13
Epoch [63/100], Step [400/600], d_loss: 0.7995, g_loss: 2.3385, D(x): 0.85, D(G(z)): 0.30
Epoch [63/100], Step [600/600], d_loss: 0.5877, g_loss: 2.5992, D(x): 0.79, D(G(z)): 0.17
Epoch [64/100], Step [200/600], d_loss: 0.6440, g_loss: 2.4815, D(x): 0.85, D(G(z)): 0.27
Epoch [64/100], Step [400/600], d_loss: 0.5453, g_loss: 3.1713, D(x): 0.78, D(G(z)): 0.13
Epoch [64/100], Step [600/600], d_loss: 0.7282, g_loss: 2.6657, D(x): 0.79, D(G(z)): 0.24
Epoch [65/

Epoch [92/100], Step [200/600], d_loss: 0.8102, g_loss: 1.4060, D(x): 0.70, D(G(z)): 0.22
Epoch [92/100], Step [400/600], d_loss: 1.0094, g_loss: 1.4781, D(x): 0.74, D(G(z)): 0.33
Epoch [92/100], Step [600/600], d_loss: 0.8112, g_loss: 1.8364, D(x): 0.77, D(G(z)): 0.31
Epoch [93/100], Step [200/600], d_loss: 0.9155, g_loss: 1.6243, D(x): 0.65, D(G(z)): 0.22
Epoch [93/100], Step [400/600], d_loss: 0.7393, g_loss: 2.2070, D(x): 0.74, D(G(z)): 0.22
Epoch [93/100], Step [600/600], d_loss: 0.8510, g_loss: 2.3589, D(x): 0.74, D(G(z)): 0.30
Epoch [94/100], Step [200/600], d_loss: 0.8031, g_loss: 1.9273, D(x): 0.74, D(G(z)): 0.24
Epoch [94/100], Step [400/600], d_loss: 0.9786, g_loss: 1.9484, D(x): 0.82, D(G(z)): 0.40
Epoch [94/100], Step [600/600], d_loss: 0.6804, g_loss: 2.1355, D(x): 0.78, D(G(z)): 0.23
Epoch [95/100], Step [200/600], d_loss: 0.7455, g_loss: 1.6581, D(x): 0.79, D(G(z)): 0.27
Epoch [95/100], Step [400/600], d_loss: 0.6516, g_loss: 2.1431, D(x): 0.80, D(G(z)): 0.25
Epoch [95/

d_loss: 0.8115 means the descriminator is able to distinguish from fake images with an accuracy of 81 %.

g_loss: 1.5439 is the generator loss. It is lower means the generator is producing better quality images.

D(x): 0.79 is the probability assigned to real images being real.

D(G(z)): 0.29 is the probability assigned to fake images being real.

### Task 2.2

In [6]:
num_epochs = 5 # Adjusted for faster comparison

# Loss function and optimizer
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(G.parameters(), lr=learning_rate)

# Function to reset gradients
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # Discriminator training
        # Real images
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Generator training
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save generated images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        utils.save_image(images, './gan_images/real_images_logistic.png')
        utils.save_image(fake_images, './gan_images/fake_images_logistic-{}.png'.format(epoch+1))

# Save the model checkpoints
torch.save(G.state_dict(), './gan_generator_logistic.pth')
torch.save(D.state_dict(), './gan_discriminator_logistic.pth')


Epoch [0/5], Step [200/600], d_loss: 0.2359, g_loss: 4.3907, D(x): 0.93, D(G(z)): 0.09
Epoch [0/5], Step [400/600], d_loss: 0.6444, g_loss: 4.9785, D(x): 0.89, D(G(z)): 0.21
Epoch [0/5], Step [600/600], d_loss: 0.4526, g_loss: 3.7107, D(x): 0.86, D(G(z)): 0.08
Epoch [1/5], Step [200/600], d_loss: 0.2238, g_loss: 4.5393, D(x): 0.93, D(G(z)): 0.05
Epoch [1/5], Step [400/600], d_loss: 0.0621, g_loss: 4.3934, D(x): 0.98, D(G(z)): 0.03
Epoch [1/5], Step [600/600], d_loss: 0.0948, g_loss: 6.6596, D(x): 1.00, D(G(z)): 0.08
Epoch [2/5], Step [200/600], d_loss: 0.1207, g_loss: 5.0339, D(x): 0.98, D(G(z)): 0.06
Epoch [2/5], Step [400/600], d_loss: 0.0456, g_loss: 5.9054, D(x): 0.98, D(G(z)): 0.01
Epoch [2/5], Step [600/600], d_loss: 0.1097, g_loss: 4.9245, D(x): 0.99, D(G(z)): 0.08
Epoch [3/5], Step [200/600], d_loss: 0.1916, g_loss: 5.2514, D(x): 0.96, D(G(z)): 0.06
Epoch [3/5], Step [400/600], d_loss: 0.1908, g_loss: 5.6847, D(x): 0.93, D(G(z)): 0.00
Epoch [3/5], Step [600/600], d_loss: 0.0580

#### Original GAN Code result:
Epoch [99/100], Step [600/600], d_loss: 0.8115, g_loss: 1.5439, D(x): 0.79, D(G(z)): 0.29

d_loss: 0.8115 means the descriminator is able to distinguish from fake images with an accuracy of 81 %.

g_loss: 1.5439 is the generator loss. It is lower means the generator is producing better quality images.

D(x): 0.79 is the probability assigned to real images being real.

D(G(z)): 0.29 is the probability assigned to fake images being real.
                
#### With BCE Loss result:
Epoch [4/5], Step [600/600], d_loss: 0.0608, g_loss: 9.5112, D(x): 0.98, D(G(z)): 0.01